In [49]:
import autogen
from autogen import config_list_from_json 

In [50]:
autogen.__version__

'0.2.2'

In [51]:
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")
USE_COHERE = False
if USE_COHERE:
    config_list[0]['model'] = 'command-nightly'

In [52]:
llm_config = dict(config_list=config_list,
                  timeout=120,
                  temperature=0.0)

## Tech Team Agents

In [53]:
priming = 'Think step by step'

user_proxy = autogen.UserProxyAgent(
    name='CEO',
    system_message="CEO. Interacts with Product Manager to discuss product requirements. Plan execution needs to be approved by CEO." + priming,
    code_execution_config=False,
)

product_manager = autogen.AssistantAgent(
    name='Senior Product Manager',
    system_message='You are a Senior Product Manager. You have vast knowledge about product development. You understand product requirements and are able to write excellent requirement documentation. When writing requirement documentation, you start with the value proposition, then user stories for the list of features, and then the acceptance criteria as well as the definition of done. You will also delegate additional research task to the UI/UX researcher. Always asks the CEO for approval after you have an initial draft. If the draft is rejected, you will iterate over them by asking questions to the UI/UX Researcher. Once it is approved, discuss with the Engineer on the technical implementation. Always check with them if the requirements makes sense and it can be achieved technically. Once the engineer has come up with the solution, discuss with the Data Engineer on what are the useful metrics to track, and why.' + priming,
    llm_config=llm_config,
    code_execution_config=False
)

ui_ux_researcher = autogen.AssistantAgent(
    name='Senior UI/UX Researcher',
    system_message='You are a Senior UI/UX Researcher. You emphatize with the users and understands how to create the best experience for users. You know how to conduct proper user research and are able to communicate the challenges to the Product Manager. When discussing with the Product Manager, be critical with your thoughts and clarify the requirements. Do not agree all the time with the Product Manager if the requirement does not make sense.' + priming,
    llm_config=llm_config,
    code_execution_config=False
)


engineer = autogen.AssistantAgent(
    name="10x Engineer",
    system_message='You are a 10x enginer. You have vast experienced in Software Development, and are able to design the best solution given the tradeoffs. When given a requirement, do not jump straight into the code. Clarify the requirements first. Then start with an initial architecture and document the process. You write your documentation in Markdown. Once you are clear on the requirements, write just enough code and ask the executor to run it. Always verify that the code works.' + priming,
    llm_config=llm_config,
    code_execution_config=False
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    llm_config=llm_config,
    code_execution_config={"last_n_messages": 3, "work_dir": "code"},
)


data_analyst = autogen.AssistantAgent(
    name="Senior Data Analyst",
    llm_config=llm_config,
    system_message='You are a Senior Data Analyst. You have vast amount of knowledge on product metrics and what adds value to the business. When approached by the Product Manager, always check what features are available, then suggests the metrics that will add value to the business.' + priming,
)


groupchat = autogen.GroupChat(agents=[user_proxy, product_manager, engineer, data_analyst, executor, ui_ux_researcher], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [54]:
user_proxy.initiate_chat(
    manager,
    message="we want to be visionary in the recruitment industry. suggest some features that could differentiate us",
)

CEO (to chat_manager):

we want to be visionary in the recruitment industry. suggest some features that could differentiate us

--------------------------------------------------------------------------------


BadRequestError: failed to send message: googleapi: Error 400: